# Alpaca-py options trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/options-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with options trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for options as an example.

In [1]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

api_key = None
secret_key = None

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables

trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

In [2]:
import os

if api_key is None:
    api_key = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    secret_key = os.environ.get('ALPACA_SECRET_KEY')

In [3]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 5.0 MB/s eta 0:00:00


In [4]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.option import OptionDataStream

from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest
)
from alpaca.trading.requests import (
    GetOptionContractsRequest,
    GetAssetsRequest,
    MarketOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus
)
from alpaca.common.exceptions import APIError

In [5]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [11]:
# check version of alpaca-py
alpaca.__version__

'0.39.1'

# Trading Client

In [63]:
# API_KEY = "Alpaca's Trading API Key (Paper Account)"
# API_SECRET = "Alpaca's Trading API Secret Key (Paper Account)"

# A safe approach to setting up API credentials for Alpaca (Assume you run this notebook in Google Colab)
# Add your key to Colab Secrets. Add your API key to the Colab Secrets manager to securely store it
from google.colab import userdata
API_KEY = userdata.get('ALPACA_API_KEY')
API_SECRET = userdata.get('ALPACA_SECRET_KEY')
BASE_URL = None
## We use paper environment for this example
PAPER = False # Please do not modify this. This example is for paper trading only.

# Initialize Alpaca clients
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)
option_historical_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET, url_override=BASE_URL)


# Below are the variables for developing this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None
# check trading account
# There are trhee new columns in the account object:
# - options_buying_power
# - options_approved_level
# - options_trading_level
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': '691827622',
    'accrued_fees': '0',
    'buying_power': '100',
    'cash': '100',
    'created_at': datetime.datetime(2023, 7, 12, 17, 57, 39, 727480, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100',
    'id': UUID('d0aef497-b3db-45d6-912c-fd4aa4d54d2c'),
    'initial_margin': '0',
    'last_equity': '0',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '1',
    'non_marginable_buying_power': '100',
    'options_approved_level': 3,
    'options_buying_power': '0',
    'options_trading_level': 3,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '100',
    'regt_buying_power': '100',
    'short_market_value': '0',
    'shorting_enabled': False,
    'sma': '0',
    'st

In [64]:
# API_KEY = "Alpaca's Trading API Key (Paper Account)"
# API_SECRET = "Alpaca's Trading API Secret Key (Paper Account)"

# A safe approach to setting up API credentials for Alpaca (Assume you run this notebook in Google Colab)
# Add your key to Colab Secrets. Add your API key to the Colab Secrets manager to securely store it
from google.colab import userdata
API_KEY = userdata.get('ALPACA_API_KEY2')
API_SECRET = userdata.get('ALPACA_SECRET_KEY2')
BASE_URL = None
## We use paper environment for this example
PAPER = True # Please do not modify this. This example is for paper trading only.

# Initialize Alpaca clients
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)
option_historical_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET, url_override=BASE_URL)


# Below are the variables for developing this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None
# check trading account
# There are trhee new columns in the account object:
# - options_buying_power
# - options_approved_level
# - options_trading_level
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3XF6T9O8HT',
    'accrued_fees': '0',
    'buying_power': '101.78',
    'cash': '101.78',
    'created_at': datetime.datetime(2025, 4, 1, 15, 17, 23, 110313, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 2,
    'daytrading_buying_power': '0',
    'equity': '101.78',
    'id': UUID('795f2bf2-2d4b-4b8e-be6d-0d3b843a7778'),
    'initial_margin': '0',
    'last_equity': '100',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '1',
    'non_marginable_buying_power': '101.78',
    'options_approved_level': 3,
    'options_buying_power': '101.78',
    'options_trading_level': 3,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '101.78',
    'regt_buying_power': '101.78',
    'short_market_value': '0',
    'shorting_enabled': Fal

In [65]:
# API_KEY = "Alpaca's Trading API Key (Paper Account)"
# API_SECRET = "Alpaca's Trading API Secret Key (Paper Account)"

# A safe approach to setting up API credentials for Alpaca (Assume you run this notebook in Google Colab)
# Add your key to Colab Secrets. Add your API key to the Colab Secrets manager to securely store it
from google.colab import userdata
API_KEY = userdata.get('ALPACA_API_KEY3')
API_SECRET = userdata.get('ALPACA_SECRET_KEY3')
BASE_URL = None
## We use paper environment for this example
PAPER = True # Please do not modify this. This example is for paper trading only.

# Initialize Alpaca clients
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)
option_historical_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET, url_override=BASE_URL)


# Below are the variables for developing this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None
# check trading account
# There are trhee new columns in the account object:
# - options_buying_power
# - options_approved_level
# - options_trading_level
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3WPUIAMVDK',
    'accrued_fees': '0',
    'buying_power': '128146.04',
    'cash': '32036.51',
    'created_at': datetime.datetime(2025, 2, 21, 17, 29, 5, 985458, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 7,
    'daytrading_buying_power': '128146.04',
    'equity': '32036.51',
    'id': UUID('e0d78870-f544-4dfe-9174-7a740745ec6e'),
    'initial_margin': '0',
    'last_equity': '32036.51',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '4',
    'non_marginable_buying_power': '32036.51',
    'options_approved_level': 3,
    'options_buying_power': '32036.51',
    'options_trading_level': 3,
    'pattern_day_trader': True,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '32036.51',
    'regt_buying_power': '64073.02',
    'short_market_value': '0',


In [29]:
# check account configuration
# - we have new field `max_options_trading_level`
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [33]:
# get list of assets which are options enabled
# - we can filter assets by `options_enabled` attribute
# - asset object has `options_enabled` attribute if it is options enabled
req = GetAssetsRequest(
  attributes = "options_enabled"
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': False,
     'id': UUID('6496d8d5-2319-4ba9-aa9e-67d3e5f61611'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'DERMTECH INC Common Stock',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'DMTKQ',
     'tradable': False},
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': False,
     'id': UUID('86232fb3-8c7d-445e-acba-36d9b05c3e8f'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'Enthusiast Gaming

In [34]:
# get list of options contracts for the given underlying symbol (e.g. SPY,AAPL)
# - get_option_contracts() is a new method to get list of options contracts
# - in this example, we get 2 options contracts for SPY,AAPL
# - you can continue to fetch options contracts by specifying page_token from next_page_token of response
underlying_symbols = ["SPY", "AAPL"]
req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,               # specify underlying symbols
    status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
    expiration_date = None,                                # specify expiration date (specified date + 1 day range)
    expiration_date_gte = None,                            # we can pass date object
    expiration_date_lte = None,                            # or string (YYYY-MM-DD)
    root_symbol = None,                                    # specify root symbol
    type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT)
    style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
    strike_price_gte = None,                               # specify strike price range
    strike_price_lte = None,                               # specify strike price range
    limit = 2,                                             # specify limit
    page_token = None,                                     # specify page token
)
res = trade_client.get_option_contracts(req)
res

{   'next_page_token': 'Mg==',
    'option_contracts': [   {   'close_price': '119.1',
                                'close_price_date': datetime.date(2025, 3, 31),
                                'expiration_date': datetime.date(2025, 4, 4),
                                'id': '0133a2da-b2e6-4061-8cc0-6a534ffac42a',
                                'name': 'AAPL Apr 04 2025 100 Call',
                                'open_interest': '10',
                                'open_interest_date': datetime.date(2025, 3, 28),
                                'root_symbol': 'AAPL',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 100.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'AAPL250404C00100000',
                                'tradable': True,
                                'type': <Contr

In [35]:
# continue to fetch option contracts if there is next_page_token in response
if res.next_page_token is not None:
    req = GetOptionContractsRequest(
        underlying_symbols = underlying_symbols,               # specify underlying symbols
        status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
        expiration_date = None,                                # specify expiration date (specified date + 1 day range)
        expiration_date_gte = None,                            # we can pass date object
        expiration_date_lte = None,                            # or string (YYYY-MM-DD)
        root_symbol = None,                                    # specify root symbol
        type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT)
        style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
        strike_price_gte = None,                               # specify strike price range
        strike_price_lte = None,                               # specify strike price range
        limit = 2,                                             # specify limit
        page_token = res.next_page_token,                      # specify page token
    )
    res = trade_client.get_option_contracts(req)
    display(res)

{   'next_page_token': 'NA==',
    'option_contracts': [   {   'close_price': '94.8',
                                'close_price_date': datetime.date(2025, 3, 19),
                                'expiration_date': datetime.date(2025, 4, 4),
                                'id': '940ae314-b8c0-4abf-a515-2b4cd267172f',
                                'name': 'AAPL Apr 04 2025 120 Call',
                                'open_interest': '13',
                                'open_interest_date': datetime.date(2025, 3, 28),
                                'root_symbol': 'AAPL',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 120.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'AAPL250404C00120000',
                                'tradable': True,
                                'type': <Contra

In [36]:
# get options contract by symbol
# - get_option_contract() is a new method to get options contract by symbol or id
symbol = res.option_contracts[0].symbol
contract = trade_client.get_option_contract(symbol)
contract

{   'close_price': '94.8',
    'close_price_date': datetime.date(2025, 3, 19),
    'expiration_date': datetime.date(2025, 4, 4),
    'id': '940ae314-b8c0-4abf-a515-2b4cd267172f',
    'name': 'AAPL Apr 04 2025 120 Call',
    'open_interest': '13',
    'open_interest_date': datetime.date(2025, 3, 28),
    'root_symbol': 'AAPL',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 120.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'AAPL250404C00120000',
    'tradable': True,
    'type': <ContractType.CALL: 'call'>,
    'underlying_asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'underlying_symbol': 'AAPL'}

In [37]:
# get options contract by id
id = res.option_contracts[0].id
contract = trade_client.get_option_contract(symbol_or_id=id)
contract

{   'close_price': '94.8',
    'close_price_date': datetime.date(2025, 3, 19),
    'expiration_date': datetime.date(2025, 4, 4),
    'id': '940ae314-b8c0-4abf-a515-2b4cd267172f',
    'name': 'AAPL Apr 04 2025 120 Call',
    'open_interest': '13',
    'open_interest_date': datetime.date(2025, 3, 28),
    'root_symbol': 'AAPL',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 120.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'AAPL250404C00120000',
    'tradable': True,
    'type': <ContractType.CALL: 'call'>,
    'underlying_asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'underlying_symbol': 'AAPL'}

In [38]:
# get put options contracts
underlying_symbols = ["SPY"]

# specify expiration date range
now = datetime.now(tz = ZoneInfo("America/New_York"))
day1 = now + timedelta(days = 1)
day60 = now + timedelta(days = 60)

req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,                     # specify underlying symbols
    status = AssetStatus.ACTIVE,                                 # specify asset status: active (default)
    expiration_date = None,                                      # specify expiration date (specified date + 1 day range)
    expiration_date_gte = day1.date(),                           # we can pass date object
    expiration_date_lte = day60.strftime(format = "%Y-%m-%d"),   # or string
    root_symbol = None,                                          # specify root symbol
    type = "put",                                                # specify option type: put
    style = ExerciseStyle.AMERICAN,                              # specify option style: american
    strike_price_gte = None,                                     # specify strike price range
    strike_price_lte = None,                                     # specify strike price range
    limit = 100,                                                 # specify limit
    page_token = None,                                           # specify page
)
res = trade_client.get_option_contracts(req)
res.option_contracts[:2]

[{   'close_price': '0.01',
     'close_price_date': datetime.date(2025, 3, 26),
     'expiration_date': datetime.date(2025, 4, 2),
     'id': 'efc44ad5-2566-4f1d-8fa1-5bfa91b09ee8',
     'name': 'SPY Apr 02 2025 400 Put',
     'open_interest': '80',
     'open_interest_date': datetime.date(2025, 3, 28),
     'root_symbol': 'SPY',
     'size': '100',
     'status': <AssetStatus.ACTIVE: 'active'>,
     'strike_price': 400.0,
     'style': <ExerciseStyle.AMERICAN: 'american'>,
     'symbol': 'SPY250402P00400000',
     'tradable': True,
     'type': <ContractType.PUT: 'put'>,
     'underlying_asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'underlying_symbol': 'SPY'},
 {   'close_price': '0.01',
     'close_price_date': datetime.date(2025, 3, 28),
     'expiration_date': datetime.date(2025, 4, 2),
     'id': 'dfaf7f6c-9743-479b-850a-56e21e54b78a',
     'name': 'SPY Apr 02 2025 405 Put',
     'open_interest': '1000',
     'open_interest_date': datetime.date(2025, 3, 28),
    

In [39]:
# get high open_interest contract
open_interest = 0
high_open_interest_contract = None
for contract in res.option_contracts:
    if (contract.open_interest is not None) and (int(contract.open_interest) > open_interest):
        open_interest = int(contract.open_interest)
        high_open_interest_contract = contract
high_open_interest_contract

{   'close_price': '0.19',
    'close_price_date': datetime.date(2025, 3, 31),
    'expiration_date': datetime.date(2025, 4, 2),
    'id': '8efaf35c-643d-4d27-a1af-8d73d8e4fba4',
    'name': 'SPY Apr 02 2025 530 Put',
    'open_interest': '6341',
    'open_interest_date': datetime.date(2025, 3, 28),
    'root_symbol': 'SPY',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 530.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'SPY250402P00530000',
    'tradable': True,
    'type': <ContractType.PUT: 'put'>,
    'underlying_asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'underlying_symbol': 'SPY'}

In [ ]:
# place buy put option order
# - we can place buy put option order same as buy stock/crypto order
req = MarketOrderRequest(
    symbol = high_open_interest_contract.symbol,
    qty = 1,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

In [40]:
# get list of orders by specifying option contract symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [high_open_interest_contract.symbol],
    limit = 2,
)
orders = trade_client.get_orders(req)
orders

[{   'asset_class': None,
     'asset_id': None,
     'canceled_at': None,
     'client_order_id': '52ad22a2-f684-4233-9b3e-492630d03700',
     'created_at': datetime.datetime(2025, 4, 1, 15, 17, 49, 901293, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'expires_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': datetime.datetime(2025, 4, 1, 15, 20, 30, 318183, tzinfo=TzInfo(UTC)),
     'filled_avg_price': '-0.04',
     'filled_qty': '1',
     'hwm': None,
     'id': UUID('43eeb069-e97e-4dbb-acf0-0fd6b779d010'),
     'legs': [   {   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
                     'asset_id': UUID('4f82efeb-0157-4ff5-aaa7-961d7e6c110d'),
                     'canceled_at': None,
                     'client_order_id': '879d6881-d531-40cf-a5d0-96699e3274cf',
                     'created_at': datetime.datetime(2025, 4, 1, 15, 17, 49, 901293, tzinfo=TzInfo(UTC)),
                     'expired_at': None,
                     'ex

In [41]:
# below cells should be done after market open otherwise there is no position for the option contract

# get positions filtered by option contract symbol
# if you do this example outside of market hours, you will see empty list
# because we have no position in this option contract
# please wait market open and run this example again
positions = trade_client.get_all_positions()
[pos for pos in positions if pos.symbol == high_open_interest_contract.symbol]

[]

In [ ]:
# get positions by symbol
trade_client.get_open_position(symbol_or_asset_id=high_open_interest_contract.symbol)


In [ ]:
# get positions by contract id
trade_client.get_open_position(symbol_or_asset_id = high_open_interest_contract.id)

In [ ]:
# close the option position
trade_client.close_position(
    symbol_or_asset_id = high_open_interest_contract.symbol,
    close_options = ClosePositionRequest(qty = "1")
)

In [ ]:
# exercise the options position
# - this method does not return anything
trade_client.exercise_options_position(
    symbol_or_contract_id = high_open_interest_contract.symbol
)

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

fyi. you can open this notebook in another window and run below cell to check trade updates.

In [42]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

ERROR:alpaca.trading.stream:error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/alpaca/trading/stream.py", line 171, in _run_forever
    await self._start_ws()
  File "/usr/local/lib/python3.11/dist-packages/alpaca/trading/stream.py", line 132, in _start_ws
    await self._auth()
  File "/usr/local/lib/python3.11/dist-packages/alpaca/trading/stream.py", line 79, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
ERROR:alpaca.trading.stream:error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/alpaca/trading/stream.py", line 171, in _run_forever
    await self._start_ws()
  File "/usr/local/lib/python3.11/dist-packages/alpaca/trading/stream.py", line 132, in _start_ws
    await self._auth()
  File "/usr/local/lib/python3.11/dist-packages/alpaca/trading/stream.py", line

keyboard interrupt, bye


# Market Data (Historical)

In [43]:
# setup option historical data client
option_historical_data_client = OptionHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

ValueError: You must supply a method of authentication

In [44]:
# get options historical bars by symbol
req = OptionBarsRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    timeframe = TimeFrame(amount = 1, unit = TimeFrameUnit.Hour),   # specify timeframe
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                                # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_bars(req).df

open  high  low  close  volume  \
symbol             timestamp                                                   
SPY250402P00530000 2025-03-27 18:00:00+00:00   0.1  0.11  0.1   0.11   270.0   
                   2025-03-27 19:00:00+00:00   0.1  0.10  0.1   0.10    10.0   

                                              trade_count      vwap  
symbol             timestamp                                         
SPY250402P00530000 2025-03-27 18:00:00+00:00          7.0  0.106667  
                   2025-03-27 19:00:00+00:00          3.0  0.100000

In [45]:
# get options historical trades by symbol
req = OptionTradesRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end=None,                                                     # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_trades(req).df

exchange  price  size  \
symbol             timestamp                                                
SPY250402P00530000 2025-03-27 17:26:18.639359+00:00        M   0.13  27.0   
                   2025-03-27 17:26:18.639359+00:00        M   0.13  10.0   

                                                    conditions  
symbol             timestamp                                    
SPY250402P00530000 2025-03-27 17:26:18.639359+00:00          I  
                   2025-03-27 17:26:18.639359+00:00          I

In [ ]:
# get options exchange codes
option_historical_data_client.get_option_exchange_codes()

In [ ]:
# get option latest quote by symbol
req = OptionLatestQuoteRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_quote(req)

In [46]:
# get option latest trade by symbol
req = OptionLatestTradeRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_trade(req)

{'SPY250402P00530000': {   'conditions': 'j',
     'exchange': 'D',
     'id': None,
     'price': 0.08,
     'size': 1.0,
     'symbol': 'SPY250402P00530000',
     'tape': None,
     'timestamp': datetime.datetime(2025, 4, 1, 16, 52, 55, 606357, tzinfo=TzInfo(UTC))}}

In [47]:
# get option snapshot by symbol
req = OptionSnapshotRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_snapshot(req)

{'SPY250402P00530000': {   'greeks': {   'delta': -0.0131,
                   'gamma': 0.0024,
                   'rho': -0.0002,
                   'theta': -0.2365,
                   'vega': 0.0099},
     'implied_volatility': 0.4795,
     'latest_quote': {   'ask_exchange': 'W',
                         'ask_price': 0.09,
                         'ask_size': 667.0,
                         'bid_exchange': 'E',
                         'bid_price': 0.04,
                         'bid_size': 1029.0,
                         'conditions': ' ',
                         'symbol': 'SPY250402P00530000',
                         'tape': None,
                         'timestamp': datetime.datetime(2025, 4, 1, 17, 17, 40, 972171, tzinfo=TzInfo(UTC))},
     'latest_trade': {   'conditions': 'j',
                         'exchange': 'D',
                         'id': None,
                         'price': 0.08,
                         'size': 1.0,
                         'symbol': 'SPY250

In [ ]:
# get option chain by underlying_symbol
req = OptionChainRequest(
    underlying_symbol = high_open_interest_contract.underlying_symbol,
)
option_historical_data_client.get_option_chain(req)

# Market Data (Stream)

In [48]:
option_data_stream_client = OptionDataStream(api_key, secret_key, url_override = option_stream_data_wss)

async def option_data_stream_handler(data):
    print(data)

symbols = [
    high_open_interest_contract.symbol,
]

option_data_stream_client.subscribe_quotes(option_data_stream_handler, *symbols)
option_data_stream_client.subscribe_trades(option_data_stream_handler, *symbols)

option_data_stream_client.run()

ERROR:alpaca.data.live.websocket:error during websocket communication: auth failed
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/alpaca/data/live/websocket.py", line 343, in _run_forever
    await self._start_ws()
  File "/usr/local/lib/python3.11/dist-packages/alpaca/data/live/websocket.py", line 135, in _start_ws
    await self._auth()
  File "/usr/local/lib/python3.11/dist-packages/alpaca/data/live/websocket.py", line 126, in _auth
    raise ValueError(msg[0].get("msg", "auth failed"))
ValueError: auth failed
ERROR:alpaca.data.live.websocket:error during websocket communication: auth failed
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/alpaca/data/live/websocket.py", line 343, in _run_forever
    await self._start_ws()
  File "/usr/local/lib/python3.11/dist-packages/alpaca/data/live/websocket.py", line 135, in _start_ws
    await self._auth()
  File "/usr/local/lib/python3.11/dist-packages/alpaca/data/live/web

keyboard interrupt, bye


KeyboardInterrupt: 